In [1]:
from kcorrect.kcorrect import Kcorrect
import pandas as pd
import numpy as np
from astropy.cosmology import FlatLambdaCDM

path = '/home/gimena/Documents/hsc/scripts/add_features/'  #of this file


# Galactic extintion correction is not done in this script!!

In [12]:
# import kcorrect
# sorted(kcorrect.response.all_responses())

In [2]:
filter_files = [# list of files with response functions (from https://hsc.mtk.nao.ac.jp/ssp/survey/#filters_and_depths)
    'hsc_g_v2018',
    'hsc_r2_v2018',
    'hsc_i2_v2018',
    'hsc_z_v2018',
    'hsc_y_v2018'
]

table_names = ['GFILTER', 'RFILTER', 'IFILTER', 'ZFILTER', 'YFILTER']

# convert them to yanny parameter files...
# Kcorrect does have some response functions for subaru, but it's missing the Y filter. Also, the i and r filters were changed in 2016 (see HSC-SSP DR2 note).
responses = []
for fil,nam in zip(filter_files,table_names):
    f = pd.read_table(f'hsc_responses_all_rev4/{fil}.dat', names = ['w','r'], delim_whitespace= True)
    f.insert(0, 'a', nam)  #name of the table
    
    with open(f'par_responses/{fil}.par', 'w') as par:
        par.write('typedef struct {\n')
        par.write(' float lambda;\n')
        par.write(' float pass;\n')
        par.write('} '+nam+';\n\n')

        dfAsString = f.to_string(header=False, index=False)
        par.write(dfAsString)
del f,fil

In [13]:
# list of par files for each filter, with ABSOLUT PATH
responses = [path + 'par_responses/' + fil for fil in filter_files] + ['wise_w1']
kc = Kcorrect(responses = responses, abcorrect= True, redshift_range= [0., 3.], nredshift = 5000, cosmo= FlatLambdaCDM(H0=70, Om0=0.3))

In [15]:
df = pd.read_csv('../../clean-HSC-unWISE-W01.csv')
mags = ['g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag', 'W1']
errs = ['g_cmodel_magsigma', 'r_cmodel_magsigma', 'i_cmodel_magsigma', 'z_cmodel_magsigma', 'y_cmodel_magsigma', 'W1_err']

In [16]:
# convert the magnitudes to "maggies"
# maggies are a linear flux density unit defined as 10^{-0.4 m_AB}  where  m_AB is the AB apparent magnitude. 
# That is, 1 maggie is the flux density in Janskys divided by 3631. 

# https://www.sdss3.org/dr8/algorithms/magnitudes.php
# http://wiki.ipb.ac.rs/index.php/Astro_links

maggies = pd.DataFrame()
ivars = pd.DataFrame()
for m,e in zip(mags, errs):
    maggies[m] = [10**(-0.4 * mm) for mm in df[m].values]     # 10^[-0.4 * m]
    ivars[m]=[1./(0.4*np.log(10.)*maggie*ee)**2 for ee,maggie in zip(df[e].values, maggies[m].values)]   # 1. / [0.4 * ln(10) * maggie * m_err]**2

In [17]:
coeffs = kc.fit_coeffs(redshift = df['phot_z'].values, maggies = maggies.values, ivar = ivars.values)  
# the docs says that it transforms maggies to the ab system.

# “NNLS quitting on iteration count.” 
# This indicates that the default number of iterations for scipy.optimize.nnls was not enough. 
# Under these conditions, this code tries a much larger number of iterations. If that still fails, you will receive a traceback.

In [18]:
abs_mags = kc.absmag(redshift = df['phot_z'].values, maggies = maggies.values, ivar = ivars.values, coeffs = coeffs)
abs_mags

array([[-21.41873986, -21.73913808, -21.92210614, -22.00432277,
        -22.31287258, -22.12238205],
       [-22.35204024, -22.64136637, -22.74976456, -22.77762126,
        -22.9611531 , -21.91420765],
       [-20.6417381 , -21.23089985, -21.56752138, -21.71765813,
        -21.90322229, -21.36836605],
       ..., 
       [-22.30065355, -23.28703126, -23.36422655, -23.97480053,
        -23.83280113, -23.4680917 ],
       [-20.81908497, -21.38137642, -21.73959875, -21.89845585,
        -22.14693091, -21.2651881 ],
       [-20.62313805, -21.2415633 , -21.50489119, -21.70434321,
        -21.93685629, -21.28668306]])

In [22]:
for i,nam in enumerate(mags):
    df[nam + '_abs'] = abs_mags[:,i]